# Install dependencies

In [1]:
!pip install torch torchvision torchaudio
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

# GCN Link Prediction Code

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.nn import GCNConv

# Load Cora dataset
dataset = Planetoid(root="data/Cora", name="Cora")
data = dataset[0]

# Updated split for link prediction
transform = RandomLinkSplit(
    num_val=0.05,
    num_test=0.1,
    is_undirected=True,
    add_negative_train_samples=True
)
train_data, val_data, test_data = transform(data)

class GCNEncoder(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class DotProductPredictor(nn.Module):
    def forward(self, z, edge_index):
        src = z[edge_index[0]]
        dst = z[edge_index[1]]
        return (src * dst).sum(dim=1)

model = GCNEncoder(dataset.num_node_features, 64, 32)
predictor = DotProductPredictor()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def get_loss(z, edge_label_index, edge_label):
    pred = predictor(z, edge_label_index)
    return F.binary_cross_entropy_with_logits(pred, edge_label.float())

@torch.no_grad()
def evaluate(data, z):
    pred = torch.sigmoid(predictor(z, data.edge_label_index))
    labels = data.edge_label

    from sklearn.metrics import roc_auc_score, average_precision_score

    auc = roc_auc_score(labels.cpu(), pred.cpu())
    ap = average_precision_score(labels.cpu(), pred.cpu())
    return auc, ap

# Training
for epoch in range(1, 201):
    model.train()
    optimizer.zero_grad()

    z = model(train_data.x, train_data.edge_index)
    loss = get_loss(
        z,
        train_data.edge_label_index,
        train_data.edge_label
    )

    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        model.eval()
        z = model(train_data.x, train_data.edge_index)

        val_auc, val_ap = evaluate(val_data, z)

        print(f"Epoch {epoch:03d} | Loss {loss:.4f} | Val AUC {val_auc:.4f} | Val AP {val_ap:.4f}")

# Test
model.eval()
z = model(train_data.x, train_data.edge_index)

test_auc, test_ap = evaluate(test_data, z)

print("Test AUC:", test_auc)
print("Test AP:", test_ap)


Processing...
Done!


Epoch 020 | Loss 0.4831 | Val AUC 0.8496 | Val AP 0.8483
Epoch 040 | Loss 0.3445 | Val AUC 0.8612 | Val AP 0.8588
Epoch 060 | Loss 0.2359 | Val AUC 0.8287 | Val AP 0.8232
Epoch 080 | Loss 0.1351 | Val AUC 0.7888 | Val AP 0.7893
Epoch 100 | Loss 0.0565 | Val AUC 0.7788 | Val AP 0.7772
Epoch 120 | Loss 0.0166 | Val AUC 0.7684 | Val AP 0.7461
Epoch 140 | Loss 0.0049 | Val AUC 0.7697 | Val AP 0.7393
Epoch 160 | Loss 0.0021 | Val AUC 0.7663 | Val AP 0.7296
Epoch 180 | Loss 0.0011 | Val AUC 0.7658 | Val AP 0.7266
Epoch 200 | Loss 0.0007 | Val AUC 0.7624 | Val AP 0.7197
Test AUC: 0.7543900708964494
Test AP: 0.7107995624121214
